In [1]:
import pandas as pd
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64


# --- Reproducible key generation ---
def generate_key_from_password(password: str, salt: bytes = b'fixed_salt_12345678'):
    """
    Generate a deterministic Fernet key from a password and static salt.
    """
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100_000,
        backend=default_backend()
    )
    return base64.urlsafe_b64encode(kdf.derive(password.encode()))



# --- Decryption to float64 DataFrame ---
def decrypt_column(encrypted_series, password, salt=b'fixed_salt_12345678', column_name="Decrypted"):
    """
    Decrypt values using the same password and salt.
    Returns a DataFrame with dtype float64.
    """
    key = generate_key_from_password(password, salt)
    fernet = Fernet(key)
    decrypted_values = encrypted_series.apply(lambda x: float(fernet.decrypt(x.encode()).decode()))
    return pd.DataFrame({column_name: decrypted_values}, dtype='float64')




def collect(rows_to_collect, github_raw_csv_url):
    """
    Collect specific rows from a CSV file hosted on GitHub.

    Parameters:
    - rows_to_collect (list of int): List of row indices to collect.
    - github_raw_csv_url (str): Raw GitHub URL pointing to the CSV file.

    Returns:
    - pandas.DataFrame: DataFrame containing only the selected rows.
    """
    # Load the CSV from GitHub
    df = pd.read_csv(github_raw_csv_url)

    # Get the rows at the specified indices
    df_selected = df.iloc[rows_to_collect]

    password = "clubes_de_ciencia_2025"
    salt = b"fixed_salt_12345678"

    decrypted = decrypt_column(df['Encrypted_y1'], password, salt, column_name='y1')

    df_selected['y1']=decrypted 
    df_selected=df_selected.drop('Encrypted_y1', axis=1)


    return df_selected

In [2]:
selected_rows=collect([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],'https://raw.githubusercontent.com/rafaelespinosacastaneda/ClubesDeCiencia2025/main/encrypted_data.csv')
print(selected_rows)

    Unnamed: 0  minimum CN_VoronoiNN  maximum CN_VoronoiNN  \
0            0              1.448237              0.336530   
1            1             -0.608465              0.752956   
2            2              1.479518              0.366708   
3            3             -0.069548             -0.383023   
4            4             -0.069548             -0.383023   
5            5             -0.914939             -0.035671   
6            6             -0.593104              0.471065   
7            7              0.112868              0.297396   
8            8             -0.069548             -0.383023   
9            9              1.479518              0.366708   
10          10             -0.852513             -1.883083   
11          11             -0.199682             -1.134426   
12          12             -0.962045             -1.988752   
13          13              0.858707              1.130487   
14          14             -0.069548             -0.383023   
15      

C:\Users\rafae\AppData\Local\Temp\ipykernel_12752\4187307184.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['y1']=decrypted
